In [1]:
import os
import autogen
from typing import List, Dict, Any
from dotenv import load_dotenv

from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
import autogen

load_dotenv()

config_list_file_name = ".config_list"

gemini_config_list = config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper_review",
        "use_docker": False,
    }

llm_config = {
    "config_list": gemini_llm_config,
    "temperature": 0.2,
    "seed": 42,
}

# Create the assistant agent
assistant = autogen.AssistantAgent(
    name="Research_Assistant",
    llm_config=gemini_llm_config,
    system_message="""You are a research assistant that helps with creating technical reviews.
You will be given tasks to perform sequentially. Follow these steps carefully:
1. Create a markdown file for the technical review
2. Search for relevant publications using Semantic Scholar
3. Add titles, authors, abstracts, and URLs to the review
4. Create a comprehensive summary of the papers
5. Generate and add references to the review

Always check if previous steps have been completed before moving to the next step.
"""
)

# Create the user proxy agent to execute code
user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "research_workspace",
        "use_docker": False,
    }
)

def generate_technical_review(topic: str):
    """Initialize the conversation between agents to generate a technical review."""
    
    user_proxy.initiate_chat(
        assistant,
        message=f"""Create a technical review for the topic: '{topic}'. 
Follow these steps in sequence:

1. Create a file named 'technical_review.md'
2. Find 5 top cited relevant publications for the topic '{topic}' using the semanticscholar Python library
    - IMPORTANT: search_paper API return PaginatedResults so it might got stuck in the loop, handle it carefully.
    - You should break the loop when count is 5
3. Append a title in the 'technical_review.md' for this topic
4. Append all 5 publications' title, authors, abstract, and URL in 'technical_review.md'
5. Read all the paper abstracts and make a comprehensive summary of the works
6. Append the summary into 'technical_review.md'
7. Generate references and append them in 'technical_review.md'

Let's proceed step by step. Please complete each task before moving to the next one.
"""
    )

# Example usage
if __name__ == "__main__":
    # Create the workspace directory if it doesn't exist
    if not os.path.exists("research_workspace"):
        os.makedirs("research_workspace")
    
    # Run the review generation
    topic = "reinforcement learning from human feedback"
    generate_technical_review(topic)

User_Proxy (to Research_Assistant):

Create a technical review for the topic: 'reinforcement learning from human feedback'. 
Follow these steps in sequence:

1. Create a file named 'technical_review.md'
2. Find 5 top cited relevant publications for the topic 'reinforcement learning from human feedback' using the semanticscholar Python library
    - IMPORTANT: search_paper API return PaginatedResults so it might got stuck in the loop, handle it carefully.
    - You should break the loop when count is 5
3. Append a title in the 'technical_review.md' for this topic
4. Append all 5 publications' title, authors, abstract, and URL in 'technical_review.md'
5. Read all the paper abstracts and make a comprehensive summary of the works
6. Append the summary into 'technical_review.md'
7. Generate references and append them in 'technical_review.md'

Let's proceed step by step. Please complete each task before moving to the next one.


---------------------------------------------------------------